In [2]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [3]:
med_income_file = "Resources/household_median_income_2017.csv"
#imported first 52 rows, last row was NaN
med_income_df = pd.read_csv(med_income_file, nrows=52)
med_income_df.head()

State   2017   2016   2015   2014   2013  2013.1   2012   2011  \
0  United States  61372  60309  58476  55613  56479   54744  54569  54673   
1        Alabama  51113  48237  46053  43820  49876   43616  46490  46520   
2         Alaska  72231  77351  77717  70095  76387   64439  68080  62731   
3        Arizona  61125  58328  54060  51050  55453   53335  50320  53108   
4       Arkansas  48829  46894  44282  46560  41503   42075  41735  45114   

    2010  ...   1993   1992   1991   1990   1989   1988   1987   1986   1985  \
0  55520  ...  52334  52615  53025  54621  55329  54390  53945  53309  51455   
1  46120  ...  42017  44323  42852  42607  40740  39852  40848  40965  39941   
2  65178  ...  71917  71791  71482  71686  68919  66133  68791  67138  75777   
3  52839  ...  51110  50420  54101  53309  54651  52811  55369  54600  52019   
4  43477  ...  38594  41015  41248  41565  41025  40299  38971  40104  38019   

    1984  
0  50511  
1  39007  
2  72912  
3  48280  
4  35320  

[5 rows x 36 columns]

In [4]:
life_expect_file = "Resources/life_expectancy_2010-2015.csv"
#imported only rows 3-55 from csv as outside rows where just notes, not part of table
life_expect_df = pd.read_csv(life_expect_file, header=2, nrows=52)

life_expect_df.head()

Location  Life Expectancy at Birth (years)  Footnotes
0  United States                              78.7        NaN
1        Alabama                              75.5        NaN
2         Alaska                              78.8        NaN
3        Arizona                              79.9        NaN
4       Arkansas                              76.0        NaN

### Create & Clean new Median Income dataframe with select columns

In [5]:
#select particular columns
new_med_income_df = med_income_df[['State','2010','2011','2012','2013','2014','2014', '2015']]

#drop duplicates (just in case)
new_med_income_df = new_med_income_df.drop_duplicates('State')

#create Median column
new_med_income_df['Median'] = new_med_income_df.median(axis=1)

# new_med_income_df.head()

In [6]:
#drop all specific year columns
new_med_income_df = new_med_income_df.drop(columns=['2010','2011','2012','2013','2014','2014', '2015'])

#rename columns
new_med_income_df = new_med_income_df.rename(columns={'State':'state','Median':'median_income'})

#set index to state
new_med_income_df = new_med_income_df.set_index('state')
new_med_income_df.head()

median_income
state                       
United States        55613.0
Alabama              46120.0
Alaska               70095.0
Arizona              52839.0
Arkansas             44282.0

### Create & Clean new Life Expectancy dataframe with select columns

In [7]:
#select particular columns
new_life_expect_df = life_expect_df[['Location', 'Life Expectancy at Birth (years)']].copy()

#rename columns
new_life_expect_df = new_life_expect_df.rename(columns={'Location':'state',
                                                'Life Expectancy at Birth (years)':'life_expectancy'})

#set index to state
new_life_expect_df = new_life_expect_df.set_index('state')

new_life_expect_df.head()

life_expectancy
state                         
United States             78.7
Alabama                   75.5
Alaska                    78.8
Arizona                   79.9
Arkansas                  76.0

### Connect to local database

In [8]:
rds_connection_string = "postgres:postgres@localhost:5432/life_income_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

###Check for tables

In [9]:
engine.table_names()

['med_income', 'life_expect']

### Use pandas to load csv converted DataFrame into database
Commented out because data is already in database.

In [16]:
# new_med_income_df.to_sql(name='med_income', con=engine, if_exists='append', index=True)

In [ ]:
# new_life_expect_df.to_sql(name='life_expect', con=engine, if_exists='append', index=True)

### Confirm data has been added by querying the tables

In [18]:
print('Median Income Table')
pd.read_sql_query('select * from med_income', con=engine).head()

Median Income Table


state  median_income
0  United States        55613.0
1        Alabama        46120.0
2         Alaska        70095.0
3        Arizona        52839.0
4       Arkansas        44282.0

In [17]:
print('Life Expectancy Table')
pd.read_sql_query('select * from life_expect', con=engine).head()

Life Expectancy Table


state  life_expectancy
0  United States             78.7
1        Alabama             75.5
2         Alaska             78.8
3        Arizona             79.9
4       Arkansas             76.0

### Join Tables

In [14]:
pd.read_sql_query('SELECT med_income.state, med_income.median_income, \
                    life_expect.life_expectancy \
                    FROM med_income LEFT JOIN life_expect \
                    ON med_income.state = life_expect.state;', con=engine).head()

state  median_income  life_expectancy
0  United States        55613.0             78.7
1        Alabama        46120.0             75.5
2         Alaska        70095.0             78.8
3        Arizona        52839.0             79.9
4       Arkansas        44282.0             76.0